# Pet Prediction

## Introduction

It happens more often than not - especially to animal & pet lovers such as myself - that you come across the most beautiful dog while going for a walk in the park -- or whenever you're mindlessly scrolling Facebook for hours. This project is a digitalised manifestation of your gung ho dog breed connoisseur from around the corner. How? Through the means of **Machine Learning** (ML).
The vast majority of blood, sweat and tears spent on researching and implementing the trained model will be documented by the use of the **Jupyter Notebook** you're currently reading.
Nevertheless, to take into account the target audience and pragmatism of this implementation, I have chosen to pair the aforementioned notebook with a frontend developed in React in order to evade the convoluted process of having to manually import images into the model -- which is still an option if you're into self-loathing.

## Imports

Your usual myriad of ML libraries were imported in this project such as:
- Numpy (Arrays & matrices) 
- Pandas (Manage the arrays)
- OS (Communication between your filesystem and python)
- Matplotlib (Plotting graphs)
- Tensorflow & Keras (Machine Learning)

Other than these I have also imported some lesser known libraries, and modules especially, to adapt to the specific circumstances of this project. These being:
- Preprocessing.image (Loading in images and transforming them into tensors)
- Sklearn.metrics (Visualising & reporting the model's performance & efficiency)
- Tensorflowjs (Exporting to a Javascript compatible model)

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
import tensorflowjs as tfjs

## Setup

In order to adhere to coding conventions and to avoid confusion further on in the project I have tactically split up the setup part of my code in order clarify some decisions I've made that other developers might not inherently also chose to do.
The most notable section is the mapping section where I list every breed conform to the alphabetically stored folders with their correspondent names.

In [2]:
TRAIN = 'breeds/TRAIN'
TEST = 'breeds/TEST'
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
EPOCH_CNT = 3

dataset=[] 
name=[]
count=0
pet_dict=[]

for file in os.listdir(TRAIN):
    pet_dict+=[file]

mapping={
	"japanese_chin": 0,
	"keeshond": 1,
	"bombay": 2,
	"egyptian_mau": 3,
	"american_pit_bull_terrier": 4,
	"pug": 5,
	"bengal": 6,
	"leonberger": 7,
	"abyssinian": 8,
	"persian": 9,
	"siamese": 10,
	"british_shorthair": 11,
	"boxer": 12,
	"newfoundland": 13,
	"staffordshire_bull_terrier": 14,
	"german_shorthaired": 15,
	"pomeranian": 16,
	"american_bulldog": 17,
	"birman": 18,
	"russian_blue": 19,
	"miniature_pinscher": 20,
	"shiba_inu": 21,
	"english_setter": 22,
	"ragdoll": 23,
	"chihuahua": 24,
	"basset_hound": 25,
	"saint_bernard": 26,
	"scottish_terrier": 27,
	"sphynx": 28,
	"maine_coon": 29,
	"great_pyrenees": 30,
	"beagle": 31,
	"yorkshire_terrier": 32,
	"wheaten_terrier": 33,
	"samoyed": 34,
	"english_cocker_spaniel": 35,
	"havanese": 36,
}

## Data Cleaning & Processing



As this project is almost entirely image based there quite simply isn't a lot of cleaning, imputation or processing to be done.

In [3]:

for file in os.listdir(TRAIN):
    path=os.path.join(TRAIN,file)
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(IMAGE_HEIGHT,IMAGE_WIDTH))
        image=img_to_array(image)
        image=image/255.0
        dataset.append([image,count])     
    count=count+1
test=[]
testfile=[]

for file in os.listdir(TEST):
    path=os.path.join(TEST,file)
    image=load_img(path, grayscale=False, color_mode='rgb', target_size=(IMAGE_HEIGHT,IMAGE_WIDTH))
    image=img_to_array(image)
    image=image/255.0
    test+=[image]
    testfile+=[file]
data,labels0=zip(*dataset)
labels1=to_categorical(labels0)
labels=np.array(labels1)
data=np.array(data)
test=np.array(test)
data2=data.reshape(-1,IMAGE_HEIGHT,IMAGE_WIDTH,3)
test2=test.reshape(-1,IMAGE_HEIGHT,IMAGE_WIDTH,3)
trainx,testx,trainy,testy=train_test_split(data,labels,test_size=0.2,random_state=44)
print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

(4712, 256, 256, 3)
(1178, 256, 256, 3)
(4712, 37)
(1178, 37)


## Training Model

In order to actually train the algorithm we'll make use of [**Densely Connected Convolutional Networks**](https://arxiv.org/abs/1608.06993) (DCCN). To go about this in a comprehensive and easy to use manner, we'll be implementing said DCCN using a Keras module which we've imported in one of the former stages of this document. 

In [4]:
datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=20, zoom_range=0.2,
                             width_shift_range=0.2, height_shift_range=0.2, shear_range=0.1, fill_mode="nearest")

pretrained_model = tf.keras.applications.DenseNet121(input_shape=(
    IMAGE_HEIGHT, IMAGE_WIDTH, 3), include_top=False, weights='imagenet', pooling='avg')

pretrained_model.trainable = False

inputs = pretrained_model.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
outputs = tf.keras.layers.Dense(37, activation='softmax')(x3)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

his = model.fit(datagen.flow(trainx, trainy, batch_size=32),
                validation_data=(testx, testy), epochs=EPOCH_CNT)

Epoch 1/3
148/148 [==============================] - 363s 2s/step - loss: 2.1823 - accuracy: 0.4213 - val_loss: 0.8098 - val_accuracy: 0.7733
Epoch 2/3
148/148 [==============================] - 412s 3s/step - loss: 1.0581 - accuracy: 0.6891 - val_loss: 0.5222 - val_accuracy: 0.8362
Epoch 3/3
 53/148 [=========>....................] - ETA: 3:27 - loss: 0.8569 - accuracy: 0.7321

KeyboardInterrupt: 

## Model Fitness

In order to fathom the efficiency and as to how our newly born model performs we will be using a couple of built-in modules and functions from the Sklearn package. 
Previously the trained model is stored in the "his" variable. After going through the various learning cycles its accuracy and loss data are then stored in sepperate new variables and are then plotted in the next section. 

In [ ]:
y_pred = model.predict(testx)
pred = np.argmax(y_pred, axis=1)
ground = np.argmax(testy, axis=1)
print(classification_report(ground, pred))

get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

## Accuracy

In [ ]:
epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

## Loss

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

## Proof testing

This section is quite possibly the most gratifying part of this entire project. Sometimes at least. 
Firstly, we declare the reverse mapping of breeds (see #Setup) to then map the different representations of breeds - using integers - to their respective indeces. A random is then loaded in and passed to the model.

In [ ]:
reverse_mapping={
    0: "abyssinian",
    1: "american_bulldog",
    2: "american_pit_bull_terrier",
    3: "basset_hound",
    4: "beagle",
    5: "bengal",
    6: "birman",
    7: "bombay",
    8: "boxer",
    9: "british_shorthair",
    10: "chihuahua",
    11: "egyptian_mau",
    12: "english_cocker_spaniel",
    13: "english_setter",
    14: "german_shorthaired",
    15: "great_pyrenees",
    16: "havanese",
    17: "japanese_chin",
    18: "keeshond",
    19: "leonberger",
    20: "maine_coon",
    21: "miniature_pinscher",
    22: "newfoundland",
    23: "persian",
    24: "pomeranian",
    25: "pug",
    26: "ragdoll",
    27: "russian_blue",
    28: "saint_bernard",
    29: "samoyed",
    30: "scottish_terrier",
    31: "shiba_inu",
    32: "siamese",
    33: "sphynx",
    34: "staffordshire_bull_terrier",
    35: "wheaten_terrier",
    36: "yorkshire_terrier",
  }

image=load_img("breeds/TEST/1002.jpg",target_size=(256,256))
image

# model = keras.models.load_model('models/model1.h5')


image=img_to_array(image) 
image=image/255.0
prediction_image=np.array(image)
prediction_image= np.expand_dims(image, axis=0)

def mapper(value):
    return reverse_mapping[value]

## Prediction

In [ ]:
# PREDICTION
print(prediction_image)
prediction=model.predict(prediction_image)
value=np.argmax(prediction)
move_name=mapper(value)
print(f"Prediction is {format(move_name).capitalize()}.")

## Serialisation

In order to make use of the functionality of the model it will be serialised as well as converted into a model which is able to be interpreted by the aforementioned React app. The TensorflowJS converter module will split up the model into one model.json as well as various weight files which will add their respective weights to the breeds. After this is done and dusted it is ready to be pushed straight to github in order for the raw model.json file to receive GET-requests.

In [ ]:
#Uncomment to save model & to export it to a tsjs compatible format.

model.save('models/model1.h5')
tfjs.converters.save_keras_model(model, 'models/tfjs')

## Conclusion

## Bronvermelding